# Getting the latent space of pre-trained models and predict task


## Importing the functions & libraries

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


from pretrain_neptune.get_pretrain_encoder import get_pretrain_encoder_from_modelID, get_pretrain_input_data
from pretrain.latent_task_predict import log_reg_multi_class, ridge_regression_predict


%who



[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
2024-09-20 00:33:18.780335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 00:33:18.802645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 00:33:18.809461: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 00:33:18.826561: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

DataLoader	 LabelEncoder	 TensorDataset	 accuracy_score	 f1_score	 get_pretrain_encoder_from_modelID	 get_pretrain_input_data	 imaplib	 importlib	 
label_encoder	 log_reg_multi_class	 ml_code_path	 nn	 np	 optim	 optuna	 os	 pd	 
precision_score	 random	 recall_score	 ridge_regression_predict	 roc_auc_score	 torch	 


## Importing the pretrain encoder from model ID

In [4]:
# getting the input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA'
(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=  get_pretrain_input_data(input_data_location)

X_data_all.shape, y_data_all.shape, X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

/home/leilapirhaji/mz_embed_engine/ml/pretrain_neptune/get_pretrain_encoder.py:72: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


((20548, 2736),
 (20548, 58),
 (14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

In [9]:
top_pretrain_models=['RCC-37832', 'RCC-38232', 'RCC-37783']
input_data_location='/home/leilapirhaji/PROCESSED_DATA'
pretrain_save_dir='/home/leilapirhaji/top_pretrained_models'

pretrain_model='RCC-37783'

# getting the latent space for the model
(pretrain_encoder, Z_all, Z_train, Z_val, Z_test, y_data_all, y_data_train, y_data_val, y_data_test)=get_pretrain_encoder_from_modelID(pretrain_model, input_data_location, pretrain_save_dir, ml_code_path)



/home/leilapirhaji/mz_embed_engine/ml/pretrain_neptune/get_pretrain_encoder.py:72: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)
[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-37783


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-37783/metadata


In [11]:
print (Z_all.shape, Z_train.shape, Z_val.shape, Z_test.shape, y_data_all.shape, y_data_train.shape, y_data_val.shape, y_data_test.shape)

(20548, 464) (14391, 464) (3074, 464) (3083, 464) (20548, 58) (14391, 58) (3074, 58) (3083, 58)


## Evaluating the latent Space

## Recon loss calcualtion


In [22]:
import importlib
import finetune.eval_finetune_latent_local_main
importlib.reload(finetune.eval_finetune_latent_local_main)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (f'Device: {device}')

from finetune.eval_finetune_latent_local_main import compute_losses

losses=compute_losses(pretrain_encoder, X_data_train, X_data_val, X_data_test, device)
losses

Device: cuda


{'train_recon_loss': 0.2652173936367035,
 'train_kl_loss': 0.045504480600357056,
 'val_recon_loss': 0.3058781921863556,
 'val_kl_loss': 0.041779227554798126,
 'test_recon_loss': 0.31078773736953735,
 'test_kl_loss': 0.041695065796375275}

In [25]:
import importlib
import pretrain.eval_pretrained_VAE
importlib.reload(pretrain.eval_pretrained_VAE)
from pretrain.eval_pretrained_VAE import calculating_recon_loss
(recon_loss_train, kl_loss_train, recon_loss_val, kl_loss_val, recon_loss_test, kl_loss_test)= calculating_recon_loss(pretrain_encoder, X_data_train, X_data_val, X_data_test)

recon_loss_train, kl_loss_train, recon_loss_val, kl_loss_val, recon_loss_test, kl_loss_test

(0.26547321677207947,
 0.045504480600357056,
 0.3056609332561493,
 0.041779227554798126,
 0.3100273609161377,
 0.041695065796375275)

## visulaizeing Latent space

In [ ]:
import importlib
import pretrain.get_pretrain_encoder
importlib.reload(pretrain.get_pretrain_encoder)

from pretrain.get_pretrain_encoder import visualize_latent_space_multiple_tasks



In [110]:
finetune_save_dir='/home/leilapirhaji/finetune_VAE_models'
#get only the directories in the finetune_save_dir
pretrain_model_list_file='/home/leilapirhaji/pretrained_models_to_finetune.txt'
# Read the text file into a DataFrame
pretrain_model_list = pd.read_csv(pretrain_model_list_file, header=None)[0]
pretrain_model_list = pretrain_model_list.dropna().tolist()
len(pretrain_model_list)

45

In [113]:
Z_pretrain_all

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,F088_210802_M462_205.mzML,-0.012116,-0.012744,0.015213,0.002109,0.044670,-0.039683,-0.059480,0.008705,-0.027577,...,0.012742,-0.055638,-0.009296,-0.047339,0.067312,0.002016,0.011414,-0.018108,0.021512,-0.011637
1,F088_210811_M462_101.mzML,-0.044780,0.018768,-0.009242,0.052131,-0.011590,-0.006703,-0.011710,0.036910,-0.031918,...,0.012081,-0.007349,0.032263,0.019297,0.011977,-0.028697,0.063601,0.035323,0.035963,-0.029250
2,F088_210802_M462_121.mzML,-0.037593,0.028535,-0.027732,-0.000944,0.014811,-0.059200,-0.055738,0.025544,0.002217,...,0.022528,-0.052873,0.004178,-0.005779,0.073328,0.000846,0.006965,0.025408,0.025243,-0.019008
3,F088_210808_M462_249.mzML,-0.011644,0.013144,0.032520,0.010189,0.020760,-0.010091,-0.016223,0.016085,-0.022232,...,-0.001954,-0.048695,0.005317,-0.009109,0.036391,0.016094,0.024453,0.000510,0.017594,0.011256
4,F088_210802_M462_251.mzML,-0.022890,0.004960,0.004098,-0.013768,0.014157,-0.022691,-0.028458,0.007696,0.001346,...,0.004858,-0.025887,0.001140,-0.001780,0.007980,0.025074,0.028331,0.022751,-0.015472,0.014215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20543,24678P_4x_Batch_18_C.mzML,-0.000845,0.017901,0.021039,0.006173,-0.001539,0.003555,-0.020716,0.011658,-0.013694,...,-0.003979,-0.014604,-0.002520,-0.028454,0.011305,-0.008400,0.002832,-0.018369,0.003462,-0.000630
20544,22623G_2x_Batch_12_C.mzML,-0.001608,0.001148,0.018269,0.015156,-0.004328,-0.007496,-0.030777,0.007229,-0.007556,...,-0.023694,-0.016688,0.004787,-0.019279,0.005315,-0.008436,0.001843,-0.020471,0.007727,-0.003372
20545,22375N_2x_Batch_11_A.mzML,0.001526,-0.005188,0.023454,-0.006104,0.003969,-0.010880,-0.019838,0.004644,-0.021066,...,0.003956,-0.024900,0.003625,-0.004619,0.001681,0.014717,0.024329,-0.020768,0.000715,-0.006457
20546,18468S_4x_Batch_12_A.mzML,0.001379,0.012236,0.036706,-0.008700,0.008016,-0.000638,-0.022410,0.008288,-0.012591,...,-0.021116,-0.021829,0.006657,-0.023547,0.019377,0.004814,0.005469,-0.018435,0.004963,0.001694


In [ ]:
import importlib
import pretrain.get_pretrain_encoder
importlib.reload(pretrain.get_pretrain_encoder)
from pretrain.get_pretrain_encoder import get_pretrain_encoder_from_modelID, visualize_latent_space_multiple_tasks

for pretrain_modelID in pretrain_model_list:
        
    print ('pretrain_modelID:', pretrain_modelID)

    embeding_path = f'{finetune_save_dir}/{pretrain_modelID}'

    Z_pretrain_all=pd.read_csv(f'{embeding_path}/Z_all_{pretrain_modelID}.csv').iloc[:, 1:]
    Z_pretrain_train=pd.read_csv(f'{embeding_path}/Z_train_{pretrain_modelID}.csv').iloc[:, 1:]
    Z_pretrain_val=pd.read_csv(f'{embeding_path}/Z_val_{pretrain_modelID}.csv').iloc[:, 1:]
    Z_pretrain_test=pd.read_csv(f'{embeding_path}/Z_test_{pretrain_modelID}.csv').iloc[:, 1:]


    #UMAP visualization of the latent space of pretrain model
    print ('UMAP visualization of the latent space of pretrain model')
    output_path=f'{finetune_save_dir}/{pretrain_modelID}'
    visualize_latent_space_multiple_tasks( pretrain_modelID, output_path, Z_pretrain_all, Z_pretrain_train, Z_pretrain_val, Z_pretrain_test, y_data_all, y_data_train, y_data_val, y_data_test)

 